# Synchronisation TDMS - Xsens par Timestamps

Ce notebook montre comment synchroniser les données TDMS (LabVIEW + cDAQ) et Xsens (GPS) en utilisant les timestamps.

## Méthode

1. **Point de départ** : Start time Xsens (référence GPS)
2. **Point de fin** : End time TDMS
3. **Trimmer TDMS** : Enlever les données avant le start Xsens
4. **Trimmer Xsens** : Enlever les données après le end TDMS
5. **Resample Xsens** : Même nombre d'échantillons que TDMS

Cette méthode élimine automatiquement les délais de script (début + fin) et corrige le drift d'horloge.

## 📦 Imports

In [ ]:
import pandas as pd
import numpy as np
from nptdms import TdmsFile
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# Configuration pour de beaux graphiques
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (15, 8)

print("✅ Imports réussis")

## 📂 Étape 1 : Charger les données brutes

In [ ]:
# Chemins des fichiers
tdms_path = "Moto_Chicane_100.tdms"
txt_path = "Moto_Chicane_100_P1.txt"

print("📊 Chargement TDMS...")
# Charger TDMS
tdms_file = TdmsFile.read(tdms_path)
group = [g for g in tdms_file.groups() if 'P1' in g.name][0] if any('P1' in g.name for g in tdms_file.groups()) else tdms_file.groups()[0]
channel = group['Edges_RoueAR']

# Extraire timestamps et données
start_time = pd.to_datetime(channel.properties.get('wf_start_time')).tz_localize(None)
increment = channel.properties.get('wf_increment')
tdms_time = start_time + pd.to_timedelta(np.arange(len(channel[:])) * increment, unit='s')

df_tdms = pd.DataFrame({
    'timestamp': tdms_time,
    'Edges': channel[:]
})
df_tdms['Edge_Diff'] = df_tdms['Edges'].diff().fillna(0)

print(f"  ✓ TDMS: {len(df_tdms)} points")
print(f"  ✓ Start: {df_tdms['timestamp'].min()}")
print(f"  ✓ End:   {df_tdms['timestamp'].max()}")

In [ ]:
print("📊 Chargement Xsens...")
# Charger Xsens
df_txt = pd.read_csv(txt_path, sep='\t', skiprows=12)
df_txt.columns = df_txt.columns.str.strip()

# Construire timestamps à partir des colonnes UTC
req_cols = ['UTC_Year', 'UTC_Month', 'UTC_Day', 'UTC_Hour', 'UTC_Minute', 'UTC_Second', 'UTC_Nano']
df_txt = df_txt.dropna(subset=req_cols)

time_series = pd.to_datetime(df_txt[req_cols[:-1]].astype(int).rename(columns={
    'UTC_Year': 'year', 'UTC_Month': 'month', 'UTC_Day': 'day',
    'UTC_Hour': 'hour', 'UTC_Minute': 'minute', 'UTC_Second': 'second'
}))
df_txt['timestamp'] = time_series + pd.to_timedelta(df_txt['UTC_Nano'], unit='ns')

# Calculer GPS Speed
if 'Vel_N' in df_txt.columns and 'Vel_E' in df_txt.columns:
    df_txt['Vel_N'] = pd.to_numeric(df_txt['Vel_N'], errors='coerce').fillna(0)
    df_txt['Vel_E'] = pd.to_numeric(df_txt['Vel_E'], errors='coerce').fillna(0)
    df_txt['GPS_Speed'] = np.sqrt(df_txt['Vel_N']**2 + df_txt['Vel_E']**2)

# Trier
df_tdms = df_tdms.sort_values('timestamp').reset_index(drop=True)
df_txt = df_txt.sort_values('timestamp').reset_index(drop=True)

print(f"  ✓ Xsens: {len(df_txt)} points")
print(f"  ✓ Start: {df_txt['timestamp'].min()}")
print(f"  ✓ End:   {df_txt['timestamp'].max()}")

## 📏 Étape 2 : Analyser les décalages initiaux

In [ ]:
# Calculer les décalages
offset_start = (df_txt['timestamp'].min() - df_tdms['timestamp'].min()).total_seconds()
offset_end = (df_txt['timestamp'].max() - df_tdms['timestamp'].max()).total_seconds()
drift = offset_end - offset_start

print("📊 Décalages avant synchronisation:")
print(f"  Décalage au début:  {offset_start:.6f} s (script delay)")
print(f"  Décalage à la fin:  {offset_end:.6f} s (script delay + drift)")
print(f"  Drift d'horloge:    {drift:.6f} s")
print(f"\n  Échantillons en trop dans Xsens: {len(df_txt) - len(df_tdms)}")

## ✂️ Étape 3 : Définir la fenêtre de synchronisation

In [ ]:
# Point de départ = start time Xsens (référence GPS)
sync_start = df_txt['timestamp'].min()

# Point de fin = end time TDMS
sync_end = df_tdms['timestamp'].max()

# Durée de synchronisation
sync_duration = (sync_end - sync_start).total_seconds()

print("🎯 Fenêtre de synchronisation:")
print(f"  Start: {sync_start}")
print(f"  End:   {sync_end}")
print(f"  Durée: {sync_duration:.6f} secondes")

## ✂️ Étape 4 : Trimmer TDMS (enlever le début)

In [ ]:
# Garder seulement les données TDMS après le start Xsens
tdms_mask = df_tdms['timestamp'] >= sync_start
df_tdms_trimmed = df_tdms[tdms_mask].copy().reset_index(drop=True)

samples_removed_tdms = len(df_tdms) - len(df_tdms_trimmed)
time_removed_tdms = (sync_start - df_tdms['timestamp'].min()).total_seconds()

print("✂️ Trimming TDMS:")
print(f"  Échantillons supprimés: {samples_removed_tdms}")
print(f"  Temps supprimé: {time_removed_tdms:.6f} s (script delay au début)")
print(f"  TDMS trimmed: {len(df_tdms_trimmed)} points")

## ✂️ Étape 5 : Trimmer Xsens (enlever la fin)

In [ ]:
# Garder seulement les données Xsens avant le end TDMS
xsens_mask = df_txt['timestamp'] <= sync_end
df_txt_trimmed = df_txt[xsens_mask].copy().reset_index(drop=True)

samples_removed_xsens = len(df_txt) - len(df_txt_trimmed)
time_removed_xsens = (df_txt['timestamp'].max() - sync_end).total_seconds()

print("✂️ Trimming Xsens:")
print(f"  Échantillons supprimés: {samples_removed_xsens}")
print(f"  Temps supprimé: {time_removed_xsens:.6f} s (script delay fin + drift)")
print(f"  Xsens trimmed: {len(df_txt_trimmed)} points")

## 🔄 Étape 6 : Resample Xsens pour avoir le même nombre d'échantillons

In [ ]:
target_samples = len(df_tdms_trimmed)

print(f"🔄 Resampling Xsens:")
print(f"  Avant: {len(df_txt_trimmed)} échantillons")
print(f"  Cible: {target_samples} échantillons")

# Créer un nouvel index temporel uniforme
time_index = pd.date_range(start=sync_start, end=sync_end, periods=target_samples)

# Créer le DataFrame resampled
df_txt_resampled = pd.DataFrame({'timestamp': time_index})

# Interpoler GPS_Speed
if 'GPS_Speed' in df_txt_trimmed.columns:
    # Convertir timestamps en secondes depuis le début
    txt_time_sec = (df_txt_trimmed['timestamp'] - sync_start).dt.total_seconds()
    new_time_sec = (time_index - sync_start).total_seconds()
    
    # Interpoler
    f_speed = interp1d(txt_time_sec, df_txt_trimmed['GPS_Speed'], 
                       kind='linear', fill_value='extrapolate')
    df_txt_resampled['GPS_Speed'] = f_speed(new_time_sec)

print(f"  Après: {len(df_txt_resampled)} échantillons")
print(f"  ✅ Même nombre d'échantillons que TDMS!")

## ✅ Étape 7 : Vérifier l'alignement

In [ ]:
print("✅ Vérification de la synchronisation:\n")

print("📊 TDMS trimmed:")
print(f"  Échantillons: {len(df_tdms_trimmed)}")
print(f"  Start: {df_tdms_trimmed['timestamp'].min()}")
print(f"  End:   {df_tdms_trimmed['timestamp'].max()}")
print(f"  Durée: {(df_tdms_trimmed['timestamp'].max() - df_tdms_trimmed['timestamp'].min()).total_seconds():.6f} s")

print("\n📊 Xsens resampled:")
print(f"  Échantillons: {len(df_txt_resampled)}")
print(f"  Start: {df_txt_resampled['timestamp'].min()}")
print(f"  End:   {df_txt_resampled['timestamp'].max()}")
print(f"  Durée: {(df_txt_resampled['timestamp'].max() - df_txt_resampled['timestamp'].min()).total_seconds():.6f} s")

# Calculer les décalages résiduels
offset_start_final = (df_txt_resampled['timestamp'].min() - df_tdms_trimmed['timestamp'].min()).total_seconds()
offset_end_final = (df_txt_resampled['timestamp'].max() - df_tdms_trimmed['timestamp'].max()).total_seconds()

print("\n🎯 Alignement final:")
print(f"  Décalage au début: {offset_start_final:.9f} s")
print(f"  Décalage à la fin:  {offset_end_final:.9f} s")
print(f"  Différence d'échantillons: {len(df_txt_resampled) - len(df_tdms_trimmed)}")

if abs(offset_end_final) < 0.001 and len(df_txt_resampled) == len(df_tdms_trimmed):
    print("\n🎉 SYNCHRONISATION PARFAITE!")
else:
    print("\n⚠️ Vérifier les résultats")

## 📈 Étape 8 : Visualiser les données synchronisées

In [ ]:
# Créer une fenêtre de visualisation au milieu des données
center_idx = len(df_tdms_trimmed) // 2
window_samples = 4000  # 10 secondes à 400 Hz

start_idx = max(0, center_idx - window_samples)
end_idx = min(len(df_tdms_trimmed), center_idx + window_samples)

# Créer le graphique
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

# Plot 1: GPS Speed
ax1.plot(df_txt_resampled['timestamp'].iloc[start_idx:end_idx], 
         df_txt_resampled['GPS_Speed'].iloc[start_idx:end_idx], 
         'b-', linewidth=1.5, label='GPS Speed (resampled)')
ax1.set_ylabel('GPS Speed (m/s)', fontsize=12)
ax1.set_title('Données synchronisées - GPS Speed', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Plot 2: Edge_Diff
ax2.plot(df_tdms_trimmed['timestamp'].iloc[start_idx:end_idx], 
         df_tdms_trimmed['Edge_Diff'].iloc[start_idx:end_idx], 
         'r-', linewidth=1.5, label='Edge_Diff (TDMS trimmed)')
ax2.set_ylabel('Edge_Diff', fontsize=12)
ax2.set_xlabel('Time', fontsize=12)
ax2.set_title('Données synchronisées - Roue arrière', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Graphique affiché")

## 💾 Étape 9 : Sauvegarder les données synchronisées

In [ ]:
# Combiner dans un seul DataFrame
df_synced = pd.DataFrame({
    'timestamp': df_tdms_trimmed['timestamp'],
    'Edges': df_tdms_trimmed['Edges'],
    'Edge_Diff': df_tdms_trimmed['Edge_Diff'],
    'GPS_Speed': df_txt_resampled['GPS_Speed']
})

# Sauvegarder
output_file = 'Moto_Chicane_100_synchronized.csv'
df_synced.to_csv(output_file, index=False)

print(f"💾 Données synchronisées sauvegardées: {output_file}")
print(f"   {len(df_synced)} lignes × {len(df_synced.columns)} colonnes")
print(f"\n📊 Aperçu:")
df_synced.head()

## 📋 Résumé de la synchronisation

In [ ]:
print("="*80)
print("📋 RÉSUMÉ DE LA SYNCHRONISATION")
print("="*80)

print(f"\n✂️ Données supprimées:")
print(f"  TDMS (début):  {samples_removed_tdms} échantillons ({time_removed_tdms:.3f}s)")
print(f"  Xsens (fin):   {samples_removed_xsens} échantillons ({time_removed_xsens:.3f}s)")

print(f"\n🔄 Resampling:")
print(f"  Xsens: {len(df_txt_trimmed)} → {target_samples} échantillons")
print(f"  Correction de fréquence: 408 Hz → 400 Hz effectif")

print(f"\n✅ Résultat:")
print(f"  Échantillons alignés: {len(df_synced)}")
print(f"  Décalage résiduel: {abs(offset_end_final):.6f}s")
print(f"  Fichier: {output_file}")

print(f"\n🎯 Avantages de cette méthode:")
print(f"  ✓ Élimine le script delay (début + fin)")
print(f"  ✓ Corrige le clock drift automatiquement")
print(f"  ✓ Pas de dépendance à un seuil de détection")
print(f"  ✓ Reproductible pour tous les essais")
print(f"  ✓ Basée sur timestamps GPS fiables")

print("\n" + "="*80)